# 1、字符串输出解析器 StrOutParser

In [1]:
from langchain.agents.output_parsers import JSONAgentOutputParser
from langchain_community.chat_models import ChatTongyi
import dotenv
import os
from langchain_core.output_parsers import StrOutputParser, XMLOutputParser
from langchain_core.prompts import ChatPromptTemplate

dotenv.load_dotenv()

chat_model = ChatTongyi(
    model="qwen3-max",
    api_key=os.getenv("DASHSCOPE_API_KEY")
)

# 调用大模型
response = chat_model.invoke("什么是大语言模型？")
print("="*20)
print(type(response))

# 如何获取一个字符串的输出结果呢？
# 方式1：自己调用输出结果的content
print("="*20)
print(response.content)

# 方式2：使用输出解析器
str_out_parser = StrOutputParser()
str_response = str_out_parser.parse(response)
print("="*20)
print(type(str_response))
print(str_response)


<class 'langchain_core.messages.ai.AIMessage'>
大语言模型（Large Language Model，简称 LLM）是一种基于深度学习的人工智能模型，专门用于理解和生成人类语言。它通过在大量文本数据上进行训练，学习语言的统计规律、语法结构、语义关系以及世界知识，从而能够完成多种自然语言处理任务。

### 核心特点：

1. **规模庞大**：
   - 大语言模型通常包含数十亿甚至数千亿个参数（如 GPT-3 有约 1750 亿参数），这使得它们具有强大的表达能力和泛化能力。
   - 训练数据量巨大，涵盖书籍、网页、新闻、百科、代码等多种来源。

2. **基于 Transformer 架构**：
   - 当前主流的大语言模型（如 GPT、BERT、PaLM、LLaMA 等）大多采用 Transformer 神经网络结构，该结构擅长处理长距离依赖和并行计算。

3. **自回归或双向建模**：
   - 像 GPT 系列采用**自回归**方式（从左到右逐词预测下一个词），适合文本生成。
   - 像 BERT 采用**双向编码**，更适合理解上下文（如问答、分类）。

4. **通用性与零样本/少样本学习能力**：
   - 大语言模型在训练后无需针对每个任务重新训练，就能通过提示（prompt）完成多种任务，例如翻译、摘要、问答、写代码等。
   - 在没有或仅有少量示例的情况下（零样本或少样本学习），也能表现出色。

5. **生成与理解能力兼备**：
   - 不仅能“读懂”输入文本的含义，还能生成连贯、合理、有时甚至富有创意的文本。

### 典型应用：

- 智能客服与对话系统（如 ChatGPT）
- 内容创作（写文章、诗歌、剧本）
- 编程辅助（GitHub Copilot）
- 机器翻译
- 信息检索与摘要
- 教育辅导与知识问答

### 局限性：

- 可能产生“幻觉”（即生成看似合理但事实错误的内容）
- 对训练数据中的偏见敏感
- 难以保证逻辑一致性或长期记忆
- 计算资源消耗大，部署成本高

总之，大语言模型是当前人工智能在自然语言处理领域的重要突破，正在深刻改变人机交互和信息处理的方式。
<class 'langchain_core.messages.ai.AIMessage'>
con

# 2、json输出解析器 JsonOutputParser

方式1：

In [2]:
from langchain_community.chat_models import ChatTongyi
import dotenv
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

dotenv.load_dotenv()

chat_model = ChatTongyi(
    model="qwen3-max",
    api_key=os.getenv("DASHSCOPE_API_KEY")
)

chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "你是一个靠谱的{role}。"),
    ("user", "{question}")
])

prompt = chat_prompt_template.invoke(input={
    "role": "人工智能专家",
    "question": "人工智能用英文怎么说？问题用q表示，回答用a表示，返回一个JSON格式的数据"
})

response = chat_model.invoke(prompt)
print(response)

parser = JsonOutputParser()
response_1 = parser.invoke(response)
print(response_1)


content='{\n  "q": "人工智能用英文怎么说？",\n  "a": "Artificial Intelligence"\n}' additional_kwargs={} response_metadata={'model_name': 'qwen3-max', 'finish_reason': 'stop', 'request_id': '917e5842-4ae3-4b09-a3e6-8470b83984f5', 'token_usage': {'input_tokens': 41, 'output_tokens': 22, 'prompt_tokens_details': {'cached_tokens': 0}, 'total_tokens': 63}} id='run--ab6fe697-1f70-46e2-932b-31e6904e5f08-0'
{'q': '人工智能用英文怎么说？', 'a': 'Artificial Intelligence'}


方式2：

举例1：

In [3]:
parser = JsonOutputParser()

print(parser.get_format_instructions())

Return a JSON object.


举例2：

In [7]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

chat_model = ChatTongyi(
    model="qwen3-max",
    api_key=os.getenv("DASHSCOPE_API_KEY")
)

joke_query = "讲一个笑话。"

parser = JsonOutputParser()

prompt = PromptTemplate(
    template="回答用户的查询。\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | chat_model | parser
output = chain.invoke(joke_query)
print(output)

{'joke': '为什么数学老师离婚了？因为他发现另一半是负数！'}


# 3、XML输出解析器 XmlOutputParser

举例1：

In [9]:
from langchain_community.chat_models import ChatTongyi
import dotenv
import os

dotenv.load_dotenv()

chat_model = ChatTongyi(
    model="qwen3-max",
    api_key=os.getenv("DASHSCOPE_API_KEY")
)

response = chat_model.invoke("请生成周杰伦的歌单，将歌名附在<song>标签中，歌手附在<artist>标签中，返回XML格式的数据。")

print(type(response))
print(response.content)


<class 'langchain_core.messages.ai.AIMessage'>
<?xml version="1.0" encoding="UTF-8"?>
<playlist>
  <track>
    <song>晴天</song>
    <artist>周杰伦</artist>
  </track>
  <track>
    <song>七里香</song>
    <artist>周杰伦</artist>
  </track>
  <track>
    <song>青花瓷</song>
    <artist>周杰伦</artist>
  </track>
  <track>
    <song>稻香</song>
    <artist>周杰伦</artist>
  </track>
  <track>
    <song>夜曲</song>
    <artist>周杰伦</artist>
  </track>
  <track>
    <song>告白气球</song>
    <artist>周杰伦</artist>
  </track>
  <track>
    <song>简单爱</song>
    <artist>周杰伦</artist>
  </track>
  <track>
    <song>双截棍</song>
    <artist>周杰伦</artist>
  </track>
  <track>
    <song>龙卷风</song>
    <artist>周杰伦</artist>
  </track>
  <track>
    <song>菊花台</song>
    <artist>周杰伦</artist>
  </track>
</playlist>


举例2：

In [12]:
from langchain_core.output_parsers import XMLOutputParser

parser = XMLOutputParser()
parser.get_format_instructions()

'The output should be formatted as a XML file.\n1. Output should conform to the tags below.\n2. If tags are not given, make them on your own.\n3. Remember to always open and close all the tags.\n\nAs an example, for the tags ["foo", "bar", "baz"]:\n1. String "<foo>\n   <bar>\n      <baz></baz>\n   </bar>\n</foo>" is a well-formatted instance of the schema.\n2. String "<foo>\n   <bar>\n   </foo>" is a badly-formatted instance.\n3. String "<foo>\n   <tag>\n   </tag>\n</foo>" is a badly-formatted instance.\n\nHere are the output tags:\n```\nNone\n```'

In [14]:
from langchain_core.output_parsers import XMLOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_community.chat_models import ChatTongyi

chat_model = ChatTongyi(
    model="qwen3-max",
    api_key=os.getenv("DASHSCOPE_API_KEY")
)

actor_query = "生成汤姆·汉克斯的简短电影记录，使用中文回答。"

parser = XMLOutputParser()

prompt = PromptTemplate(
    template="回答用户的查询。\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | chat_model | parser

output = chain.invoke(actor_query)

print(output)

{'response': [{'actor': '汤姆·汉克斯'}, {'filmography': [{'movie': '《阿甘正传》'}, {'movie': '《拯救大兵瑞恩》'}, {'movie': '《费城故事》'}, {'movie': '《荒岛余生》'}, {'movie': '《菲利普斯船长》'}]}]}
